# LU decomposition - introduction

The [LU decomposition](https://en.wikipedia.org/wiki/LU_decomposition) was introduced by no other than [Alan Turing](https://en.wikipedia.org/wiki/Alan_Turing), in his paper [Rounding-off errors in matrix processes, The Quarterly Journal of Mechanics and Applied Mathematics, 1: 287–308. doi:10.1093/qjmam/1.1.287](http://dx.doi.org/10.1093/qjmam/1.1.287). The LU decomposition is a factorization process that describes a matrix as product  $\mathbf{A} = \mathbf{L} \mathbf{U}$, were $\mathbf{L}$ is a unit triangular matrix (i.e., all the entries of its main diagonal are equal to one) and $\mathbf{U}$ is a upper triangular matrix. In his paper, Turing showed, among other things, how the Gaussian elimination enshrines the LU decompositon.

Let's firts recall the $3 \times 3$ linear system $\mathbf{A}\mathbf{x} = \mathbf{y}$ presented in our last classes:

In [1]:
import numpy as np
from scipy.linalg import lu

In [2]:
A = np.array([[2.,1.,-1.],
              [-3.,-1.,2.],
              [-2.,1.,2.]])

In [3]:
y = np.array([8., -11., -3.])

The solution of this system is given by:

In [4]:
x = np.linalg.solve(A,y)

In [5]:
print x

[ 2.  3. -1.]


Consider the solution of this linear system by using the Gaussian elimination without pivoting, as follows:

$$
\begin{array}{ccccc}
\mathbf{A}^{(0)} = \mathbf{A} & & & \mathbf{y}^{(0)} = \mathbf{y} \\\\
\mathbf{A}^{(1)} = \left(\mathbf{I} - \mathbf{M}^{(1)}\right) \mathbf{A}^{(0)} & & &
\mathbf{y}^{(1)} = \left(\mathbf{I} - \mathbf{M}^{(1)}\right) \mathbf{y}^{(0)} \\\\
\mathbf{A}^{(2)} = \left(\mathbf{I} - \mathbf{M}^{(2)}\right) \mathbf{A}^{(1)} & & &
\mathbf{y}^{(2)} = \left(\mathbf{I} - \mathbf{M}^{(2)}\right) \mathbf{y}^{(1)}
\end{array} \: ,$$

where $\mathbf{M}^{(k)}$ is a matrix given by

$$\mathbf{M}^{(k)} = \mathbf{t}^{(k)} \otimes (\mathbf{u}^{(k)})^{\top} \: ,$$

$\mathbf{u}^{(k)}$ is a $3 \times 1$ vector with all elements equal to $0$, except the $k$th element, which is equal to $1$, and $\mathbf{t}^{(k)}$ is a $3 \times 1$ vector whose $i$th element $t_{i}^{(k)}$ is given by:

$t_{i}^{(k)} = \begin{cases} 0 & \quad \text{if } i \le k \\\\ \dfrac{a^{(k-1)}_{ik}}{a^{(k-1)}_{kk}} & \quad \text{if } i \gt k\\ \end{cases} \: ,$

where $a^{(k-1)}_{ij}$ is the $ij$ element of the matrix $\mathbf{A}^{(k-1)}$.

At the end of this algorithm, the original matrix $\mathbf{A}$ is transformed into an upper triangular matrix $\mathbf{A}^{(2)}$.

Now, let's analyze the matrices $\mathbf{M}^{(1)}$ and $\mathbf{M}^{(2)}$. They are given by:

The Gaussian elimination with partial pivoting can be implemented as follows:

    N = y.size
    C = np.hstack(np.copy(A), np.copy(y))
    D = np.identity(N)
    
    for i = 1:N-1
        
        # permutation step (computation of C tilde)
        p, C = np.permut(C, i)
        D = D[p]
        
        # assert the pivot is nonzero
        assert C[i,i] != 0., 'null pivot!'
        
        # calculate the Gauss multipliers and store them 
        # in the lower part of C
        C[i+1:,i] = C[i+1:,i]/C[i,i]
        
        # zeroing of the elements in the ith column
        C[i+1:,i+1:] = C[i+1:,i+1:] - outer(C[i+1:,i], C[i,i+1:])
        
    return C[:,:N], C[:,N]

The permutation function can be defined as follows:

    permut (C, i):
        p = [j for j in range(C.shape[0])]
        imax = i + np.argmax(np.abs(C[i:,i]))
        if imax != i:
            p[i], p[imax] = p[imax], p[i]
    return p, C[p,:]

### Exercise 14

1. Implement the Gaussian elimination with partial pivoting to calculate the equivalent triangular system. The code must use the outer product function that was implemented in a previous class. The code must receive the matrix `A` and the vector `y` and return the matrix `B` and the vector `z`. The Gauss multipliers must be stored below the diagonal of `B`.

2. Use the function implemented in a previous class for solving the equivalent (upper) triangular system `Bx = z`.

3. Use the `code-template.ipynb` to test your code against `np.dot(A,y)`.

## Inverse matrices

Sometimes, we need to calculate the inverse of a matrix. The inverse of a $N \times N$ matrix $\mathbf{A}$ is commonly represented by $\mathbf{A}^{-1}$. The inverse satisfies the following property:

$$
\begin{split}
\mathbf{A}^{-1} \mathbf{A} &= \mathbf{I} \\
\mathbf{A} \mathbf{A}^{-1} &= \mathbf{I} 
\end{split} \: ,
$$

where $\mathbf{I}$ represents the identity matrix.

The second equation presented above can be conveniently rewritten by using a column partition given by:

$$
\mathbf{A} 
\left[ \mathbf{A}^{-1}\left[ \, : \, , \, 1  \right] \cdots \mathbf{A}^{-1} \left[ \, : \, , \, N  \right]  \right] = 
\left[ \mathbf{u}_{1} \cdots \mathbf{u}_{N} \right] \: ,
$$

where $\mathbf{u}_{i}$, $i = 1, \dots, N$, is a $N \times 1$ vector with all elements equal to zero, except the $i$th element, which is equal to $1$. The vectors $\mathbf{A}^{-1}\left[ \, : \, , \, i  \right]$ and $\mathbf{u}_{i}$ represent the $i$th column of $\mathbf{A}^{-1}$ and $\mathbf{I}$, respectively. This equation can then be separated into $N$ linear systems:

$$
\begin{split}
\mathbf{A} \, \mathbf{A}^{-1} \left[ \, : \, , \, 1 \right] &= \mathbf{u}_{1} \\
\mathbf{A} \, \mathbf{A}^{-1} \left[ \, : \, , \, 2 \right] &= \mathbf{u}_{2} \\
\vdots \\
\mathbf{A} \, \mathbf{A}^{-1} \left[ \, : \, , \, N \right] &= \mathbf{u}_{N}
\end{split} \: .
$$

This equation shows that each column of the inverse matrix $\mathbf{A}^{-1}$ can be calculated by solving a linear system.

### Exercise 15

1. Use your Gauss elimination code for calculating the inverse of a matrix. The code must receive a matrix `A` and calculate its inverse, column by column.

2. Use the `code-template.ipynb` to test your code against `np.inv(A)`.